In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geopy.distance as gd

from datetime import datetime, timedelta

import pickle

In [2]:
print(datetime.now())

2021-07-21 21:09:26.546517


In [3]:
def apply_relationship(row):
    consumption_at_period = row.fuel/row.report_hours
    df_positions_2019.loc[((df_positions_2019['new_position_received_time']+timedelta(hours=(int(row.time_offset.split(":")[0])), 
                                                          minutes=(int(row.time_offset.split(":")[1]))))
                          >= row.new_report_time-timedelta(hours=row.report_hours )) 
                         & 
                         ((df_positions_2019['new_position_received_time']+timedelta(hours=(int(row.time_offset.split(":")[0])), 
                                                          minutes=(int(row.time_offset.split(":")[1]))))
                          < row.new_report_time)
                         &
                         (df_positions_2019['vessel_id']== row.vessel_id)
                             , ['time_period', 'ref_date', 'daily_vessel_id', 'fuel_consumption_average']] = [row.time_period, row.new_report_time, row.id, consumption_at_period]        

In [31]:
def apply_relationship_with_print(row):
    consumption_at_period = row.fuel/row.report_hours
    return_val_0 = timedelta(hours=(int(row.time_offset.split(":")[0])),minutes=(int(row.time_offset.split(":")[1])))
    return_val_1 = row.new_report_time-timedelta(hours=row.report_hours)
    return_val_2 = row.new_report_time 
    return_val_3 = row.vessel_id

    
    print(return_val_0)
    print(return_val_1)
    print(return_val_2)
    print(return_val_3)
    
    
    return [return_val_0, return_val_1, return_val_2, return_val_3] 
    

In [14]:
# df_positions_2019 = pd.read_csv("./data/positions_dec2019.csv")
df_positions_2019 = pd.read_csv("./data/test/ML_location_data.csv")
# df_positions_2019 = pd.read_pickle("./data/[PSV]df_positions_2019_ver2.pkl")


df_positions_2019 = df_positions_2019[(df_positions_2019['vessel_id'].isin(list_vessel_id))]
df_positions_2019["new_position_received_time"] = pd.to_datetime(df_positions_2019['position_received_time'])
df_positions_2019['new_position_received_time'] = df_positions_2019['new_position_received_time'].dt.tz_localize(None)

df_positions_2019.drop_duplicates(subset=['vessel_id', 'course', 'destination', 'draught', 'heading',
       'latitude', 'longitude', 'nav_status', 'speed', 'eta_time',
       'position_received_time', 'location', 'api_source'], inplace=True)

df_positions_2019.sort_values(by=['vessel_id', 'new_position_received_time'], inplace=True)

/home/efs/anaconda3/envs/vessel_analysis_2/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
vessel_information = pd.read_csv("./data/Vessel_information.csv")

list_vessel_id = vessel_information.loc[(vessel_information['type'] == 'PSV'), 'vessel_id'].to_list()

# df_vessel_daily_data = pd.read_csv("./data/test/ML_fuel_data_updated-No_activity.csv", sep=',')
# df_vessel_daily_data_raw = pd.read_csv("./data/test/ML_fuel_data_updated-No_activity.csv", sep=',')
df_vessel_daily_data = pd.read_excel('./data/test/Data_for_Elga.xlsx', engine='openpyxl') 
df_vessel_daily_data_raw = pd.read_excel('./data/test/Data_for_Elga.xlsx', engine='openpyxl') 

# df_vessel_daily_data['fuel'] = df_vessel_daily_data2['fuel']
# df_vessel_daily_data_raw['fuel'] = df_vessel_daily_data2['fuel']

df_vessel_daily_data["id"] = df_vessel_daily_data.index + 10000
df_vessel_daily_data_raw["id"] = df_vessel_daily_data_raw.index + 10000

df_vessel_daily_data.loc[df_vessel_daily_data['time_offset'] == 'Z', 'time_offset'] = '00:00'
df_vessel_daily_data["new_report_date"] = pd.to_datetime(df_vessel_daily_data['report_date'])

df_vessel_daily_data.sort_values(by=['vessel_id', 'new_report_date', 'time_period'], inplace=True)
df_vessel_daily_data.reset_index(drop=True, inplace=True)

for idx, row in df_vessel_daily_data.iterrows():
    if idx < 1:
        df_vessel_daily_data.loc[idx, 'new_report_time'] = row['new_report_date'] +  timedelta(hours=row['report_hours'])
    else:
        if row['time_period'] == 1:
            df_vessel_daily_data.loc[idx, 'new_report_time'] = row['new_report_date'] + timedelta(hours=row['report_hours'])
        else:
            df_vessel_daily_data.loc[idx, 'new_report_time'] = df_vessel_daily_data.iloc[idx-1]['new_report_time'] + timedelta(hours=row['report_hours'])


In [5]:
df_vessel_daily_data.vessel_id.count()

42033

In [6]:
# df_vessel_daily_data.columns
# ids = df_vessel_daily_data['id']
# df_vessel_daily_data[ids.isin(ids[ids.duplicated()])]

df_duplicates = pd.concat(g for _, g in df_vessel_daily_data.groupby(['vessel_id', 'report_date', 'time_period']) if len(g) > 1)

In [7]:
df_duplicates.count()

vessel_id          364
name               364
type               364
report_date        364
report_hours       364
time_period        364
time_offset        364
fuel               364
activity             0
activitymode         0
id                 364
new_report_date    364
new_report_time    364
dtype: int64

In [8]:
df_vessel_daily_data = df_vessel_daily_data.loc[(df_vessel_daily_data.time_offset.isna() == False)
                                                &
                                                (df_vessel_daily_data['vessel_id'].isin(list_vessel_id))
                                               ]


df_vessel_daily_data.drop_duplicates(subset=['vessel_id', 'report_date', 'time_period'], inplace=True)

In [9]:
df_vessel_daily_data_raw.id.count() - df_vessel_daily_data.id.count()

5435

In [10]:
df_vessel_daily_data.id.count()

36598

In [11]:
df_vessel_daily_data.columns

Index(['vessel_id', 'name', 'type', 'report_date', 'report_hours',
       'time_period', 'time_offset', 'fuel', 'activity', 'activitymode', 'id',
       'new_report_date', 'new_report_time'],
      dtype='object')

In [12]:
len(df_vessel_daily_data.vessel_id.unique())

23

In [13]:
df_vessel_daily_data.activity.value_counts()

Series([], Name: activity, dtype: int64)

In [15]:
df_positions_2019.vessel_id.count()

1630651

In [17]:
# df_positions_2019 = df_positions_2019.loc[(df_positions_2019.new_position_received_time <='2019-12-23')]

In [16]:
for vessel_id in list_vessel_id:
    df_positions_2019.loc[df_positions_2019['vessel_id']==vessel_id, 'prev_lon'] = df_positions_2019.loc[df_positions_2019['vessel_id']==vessel_id, 'longitude'].shift(1)
    df_positions_2019.loc[df_positions_2019['vessel_id']==vessel_id, 'prev_lat'] = df_positions_2019.loc[df_positions_2019['vessel_id']==vessel_id, 'latitude'].shift(1)
    
    df_positions_2019.loc[df_positions_2019['vessel_id']==vessel_id, 'prev_receive_time'] = df_positions_2019.loc[df_positions_2019['vessel_id']==vessel_id, 'new_position_received_time'].shift(1)

In [17]:
df_positions_2019['prev_coord'] = df_positions_2019.apply(lambda x: (x.prev_lat, x.prev_lon), axis=1)
df_positions_2019['curr_coord'] = df_positions_2019.apply(lambda x: (x.latitude, x.longitude), axis=1)

In [18]:
# Here is the calculation of mile since the last record (current_coordinate - previous_coordinate)
df_positions_2019['mile_since'] = df_positions_2019.loc[df_positions_2019['prev_lon'].notnull()].apply(
    lambda x: gd.distance(x.prev_coord, x.curr_coord).nm, axis=1) 

In [19]:
# Here is the calculation of hour since the last record (current_time - previous_time)
# Then convert to hourly format
df_positions_2019['hour_since'] = (df_positions_2019.new_position_received_time - df_positions_2019.prev_receive_time)/np.timedelta64(1, 'h')

In [20]:
# Here is the calculation of speed (mile_since - hour_since) --> nautical_mile/hour
df_positions_2019.at[(df_positions_2019['hour_since'] == 0) & (df_positions_2019['mile_since'] > 0), 'hour_since']=0.001

df_positions_2019['speed_nm'] = (df_positions_2019.mile_since / df_positions_2019.hour_since)


In [21]:
df_positions_2019.loc[(df_positions_2019['hour_since'] == 0) & (df_positions_2019['mile_since'] == 0), 'speed_nm'] = 0

In [22]:
df_vessel_daily_data[(df_vessel_daily_data['time_offset'].isna() == False)
                     &
                     (df_vessel_daily_data['vessel_id'].isin(list_vessel_id))].apply(lambda x: apply_relationship(x), axis=1)



# apply_label_combine_transit('Standby','Standby')
# apply_label_combine_transit('Transit Eco', 'TransitCombine')
# apply_label_combine_transit('Transit', 'TransitCombine')
# apply_label_combine_transit('Transit Max', 'TransitCombine')
# apply_label_combine_transit('Port', 'Port')

0        None
1        None
2        None
3        None
4        None
         ... 
42028    None
42029    None
42030    None
42031    None
42032    None
Length: 36598, dtype: object

In [23]:
list_daily_vessel_id = df_vessel_daily_data.id.to_numpy()

In [24]:
df_positions_2019.columns

Index(['id', 'vessel_id', 'course', 'destination', 'draught', 'heading',
       'latitude', 'longitude', 'nav_status', 'speed', 'eta_time',
       'position_received_time', 'location', 'api_source',
       'new_position_received_time', 'prev_lon', 'prev_lat',
       'prev_receive_time', 'prev_coord', 'curr_coord', 'mile_since',
       'hour_since', 'speed_nm', 'time_period', 'ref_date', 'daily_vessel_id',
       'fuel_consumption_average'],
      dtype='object')

In [25]:
df_aggregate_data = df_positions_2019.loc[
    (df_positions_2019['daily_vessel_id'].isna() == False), 
    ['id', 'vessel_id', 'speed','new_position_received_time',
     'prev_receive_time', 'prev_coord', 'curr_coord', 'mile_since','hour_since', 
     'speed_nm', 'fuel_consumption_average', 
     'time_period', 'ref_date', 'daily_vessel_id']].sort_values(by=['vessel_id', 'new_position_received_time'])

In [26]:
df_aggregate_data['ref_date'] = pd.to_datetime(df_aggregate_data.ref_date, errors='coerce').dt.date

In [27]:
len(df_aggregate_data.daily_vessel_id.unique())
list_daily_vessel_id_agg = df_aggregate_data.daily_vessel_id.unique()

In [28]:
list_id_not_in = df_vessel_daily_data.loc[df_vessel_daily_data['id'].isin(list_daily_vessel_id_agg) == False].id.to_numpy()
list_id_not_in

array([10010, 10191, 10291, ..., 35554, 35642, 35644])

In [29]:
df_vessel_daily_data[(df_vessel_daily_data['id'] == 35554)]

,vessel_id,name,type,report_date,report_hours,time_period,time_offset,fuel,activity,activitymode,id,new_report_date,new_report_time
41701,17445,SKANSI SJOBORG,PSV,2020-09-15,23.999666,1,+02:00,1.022264,NaN,NaN,35554,2020-09-15,2020-09-15 23:59:58.798370


In [32]:
result_list = df_vessel_daily_data[(df_vessel_daily_data['id'] == 35554)].apply(lambda x: apply_relationship_with_print(x), axis=1)
list_params = result_list.values[0]

2:00:00
2020-09-15 00:00:00
2020-09-15 23:59:58.798370
17445


In [33]:
df_positions_2019.loc[((df_positions_2019['new_position_received_time']+list_params[0])
                          >= list_params[1]) 
                         & 
                         ((df_positions_2019['new_position_received_time']+list_params[0])
                          < list_params[2])
                         &
                         (df_positions_2019['vessel_id']== list_params[3])]

,id,vessel_id,course,destination,draught,heading,latitude,longitude,nav_status,speed,...,prev_receive_time,prev_coord,curr_coord,mile_since,hour_since,speed_nm,time_period,ref_date,daily_vessel_id,fuel_consumption_average


In [34]:
df_positions_2019.loc[((df_positions_2019['new_position_received_time'])
                          >= list_params[1]) 
                         & 
                         ((df_positions_2019['new_position_received_time'])
                          < list_params[2])
                         &
                         (df_positions_2019['vessel_id']== list_params[3]), 'new_position_received_time']

Series([], Name: new_position_received_time, dtype: datetime64[ns])

In [35]:
df_positions_2019.loc[((df_positions_2019['new_position_received_time']+list_params[0])
                          >= list_params[1]-timedelta(hours=24)) 
                         & 
                         ((df_positions_2019['new_position_received_time']+list_params[0])
                          < list_params[2]+timedelta(hours=5))
                         &
                         (df_positions_2019['vessel_id']== list_params[3]), 'new_position_received_time']

1960062   2020-09-13 22:01:25.195
1960063   2020-09-13 22:06:30.402
1960064   2020-09-13 22:11:34.605
1960065   2020-09-13 22:16:37.793
1960066   2020-09-13 22:21:41.996
1960067   2020-09-13 22:26:46.196
1960068   2020-09-13 22:31:49.385
1960069   2020-09-13 22:36:53.587
1960070   2020-09-13 22:41:57.788
1960071   2020-09-13 22:47:00.976
1960072   2020-09-13 22:52:05.180
1960073   2020-09-13 22:57:09.382
1960074   2020-09-13 23:02:13.582
1960075   2020-09-13 23:07:16.770
1960076   2020-09-13 23:12:21.986
1960077   2020-09-13 23:17:26.189
1960078   2020-09-13 23:22:30.391
1960079   2020-09-13 23:27:33.577
1960080   2020-09-13 23:32:37.781
1960081   2020-09-13 23:37:41.982
1960082   2020-09-13 23:42:46.183
1960083   2020-09-13 23:47:49.374
1960084   2020-09-13 23:52:53.574
1960085   2020-09-13 23:57:57.776
1960086   2020-09-16 00:00:09.494
1960087   2020-09-16 00:05:13.696
1960088   2020-09-16 00:10:17.898
1960089   2020-09-16 00:15:21.840
1960090   2020-09-16 00:20:25.287
1960091   2020

In [36]:
df_positions_2019

,id,vessel_id,course,destination,draught,heading,latitude,longitude,nav_status,speed,...,prev_receive_time,prev_coord,curr_coord,mile_since,hour_since,speed_nm,time_period,ref_date,daily_vessel_id,fuel_consumption_average
0,44392977,88,62.000000,TRANSOCEAN NORGE,5.5,56,61.297137,2.559763,underway using engine,10.700000,...,NaT,"(nan, nan)","(61.297137, 2.5597630000000002)",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,44392756,88,68.600000,NaN,NaN,61,61.300048,2.572957,under way using engine,10.500000,...,2020-01-01 00:07:28.000,"(61.297137, 2.5597630000000002)","(61.30004833, 2.57295667)",0.420103,0.039364,10.672372,NaN,NaN,NaN,NaN
2,44395182,88,64.600000,NaN,NaN,63,61.312108,2.627665,under way using engine,10.600000,...,2020-01-01 00:09:49.709,"(61.30004833, 2.57295667)","(61.31210833, 2.627665)",1.741339,0.166792,10.440166,NaN,NaN,NaN,NaN
3,44395606,88,66.500000,NaN,NaN,60,61.323590,2.684065,under way using engine,10.200000,...,2020-01-01 00:19:50.161,"(61.31210833, 2.627665)","(61.32359, 2.684065)",1.771534,0.169123,10.474803,NaN,NaN,NaN,NaN
4,44396652,88,66.000000,TRANSOCEAN NORGE,5.5,61,61.330035,2.717613,underway using engine,10.200000,...,2020-01-01 00:29:59.005,"(61.32359, 2.684065)","(61.330034999999995, 2.717613)",1.044685,0.102776,10.164641,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2092062,87847216,17445,48.200001,NaN,0.0,NaN,72.493477,20.366852,NaN,0.603323,...,2020-12-31 23:40:57.830,"(72.4934768676758, 20.3668422698975)","(72.4934768676758, 20.3668518066406)",0.000173,0.003736,0.046292,NaN,NaN,NaN,NaN
2092063,87846812,17445,203.199997,NaN,0.0,NaN,72.493477,20.366804,NaN,0.401570,...,2020-12-31 23:41:11.281,"(72.4934768676758, 20.3668518066406)","(72.4934768676758, 20.3668041229248)",0.000865,0.080276,0.010773,NaN,NaN,NaN,NaN
2092064,87847217,17445,165.199997,NaN,0.0,NaN,72.493477,20.366842,NaN,0.327697,...,2020-12-31 23:46:00.273,"(72.4934768676758, 20.3668041229248)","(72.4934768676758, 20.3668422698975)",0.000692,0.004225,0.163763,NaN,NaN,NaN,NaN
2092065,87847218,17445,89.500000,NaN,0.0,NaN,72.493439,20.366941,NaN,0.567558,...,2020-12-31 23:46:15.482,"(72.4934768676758, 20.3668422698975)","(72.4934387207031, 20.3669414520264)",0.002919,0.084500,0.034541,NaN,NaN,NaN,NaN


In [37]:
df_aggregate_data.ref_date.value_counts()

2020-11-20    9491
2020-12-28    8929
2020-11-23    8812
2020-12-29    8721
2020-11-16    8709
              ... 
2020-08-13    2100
2020-06-12    2068
2020-07-23    2015
2020-06-13    1617
2021-01-01    1496
Name: ref_date, Length: 366, dtype: int64

In [38]:
df_aggregate_data = df_aggregate_data.groupby(by=['daily_vessel_id']).agg({'mile_since': 'sum',
                                                                           'hour_since': 'sum',
                                                                           'new_position_received_time': 'min',
                                                                           'speed': ['max', 'mean'], 
                                                                           'speed_nm': ['max', 'median', 'mean'], 
                                                                           'fuel_consumption_average': ['max', 'mean'],
                                                                           'curr_coord': ['first', 'last'],
                                                                 }).reset_index(col_level=0)

df_aggregate_data.columns = ['_'.join(col).strip() for col in df_aggregate_data.columns.values]

In [39]:
df_aggregate_data['period_total_distance'] = df_aggregate_data.apply(lambda x: gd.distance(x['curr_coord_first'],
                                                                                          x['curr_coord_last']
                                                                                          ).nm, axis=1) 

In [40]:
df_aggregate_data.loc[df_aggregate_data['period_total_distance'] == 0, 'period_total_distance'] = 0.00001
df_aggregate_data['distance_ratio'] = df_aggregate_data.mile_since_sum/df_aggregate_data.period_total_distance


In [41]:
df_aggregate_data = df_aggregate_data[['hour_since_sum',
                                                                                                             'mile_since_sum',
                                                                                                             'period_total_distance',
                                                                                                             'distance_ratio',
                                                                                                             'fuel_consumption_average_max', 
                                                                                                             'fuel_consumption_average_mean',
                                                                                                             'speed_nm_max', 'speed_nm_median', 
                                                                                                             'speed_nm_mean',
                                                                                                             'daily_vessel_id_'
                                                                                                    ]]

# df_aggregate_data.loc[df_aggregate_data['transit_type_first'].isin(['Transit Max']), 'activity_label_first'] = df_aggregate_data['transit_type_first']

# df_aggregate_data.activity_label_first.count()

In [42]:
df_aggregate_data.count()

hour_since_sum                   34243
mile_since_sum                   34243
period_total_distance            34243
distance_ratio                   34243
fuel_consumption_average_max     34243
fuel_consumption_average_mean    34243
speed_nm_max                     34243
speed_nm_median                  34243
speed_nm_mean                    34243
daily_vessel_id_                 34243
dtype: int64

In [43]:
df_aggregate_data.loc[(df_aggregate_data['daily_vessel_id_'].isin([10038
                                                                    ,10044
                                                                    ,10060
                                                                    ,10069
                                                                    ,10079
                                                                    ,10093
                                                                    ,10125,]))
                     |
                     (df_aggregate_data['daily_vessel_id_'].isin([10000
                                                                    ,10002
                                                                    ,10016
                                                                    ,10017
                                                                    ,10018
                                                                    ,10019
                                                                    ,10032
                                                                    ,10033,]))]

,hour_since_sum,mile_since_sum,period_total_distance,distance_ratio,fuel_consumption_average_max,fuel_consumption_average_mean,speed_nm_max,speed_nm_median,speed_nm_mean,daily_vessel_id_
0,24.050721,0.068762,0.001710,40.215063,0.071250,0.071250,6.597903,0.003082,0.069545,10000.0
2,8.200334,0.049021,0.022233,2.204896,0.209381,0.209381,0.153738,0.002786,0.010972,10002.0
15,8.400554,0.033457,0.000261,128.191222,0.106886,0.106886,0.114610,0.004136,0.011749,10016.0
16,23.950508,0.091190,0.001069,85.293045,0.000000,0.000000,0.391957,0.003877,0.015282,10017.0
17,24.000534,0.085203,0.000488,174.562223,0.000000,0.000000,0.406028,0.003236,0.008232,10018.0
18,10.386038,0.046939,0.000395,118.789572,0.016682,0.016682,0.141830,0.003968,0.012908,10019.0
31,19.156111,0.395337,0.083416,4.739357,0.089219,0.089219,0.870056,0.003982,0.030788,10032.0
32,21.750556,0.055093,0.000564,97.598786,0.085499,0.085499,0.091321,0.002576,0.006484,10033.0
37,3.700000,0.647678,0.027893,23.220194,0.153887,0.153887,0.336884,0.129496,0.149034,10038.0
43,3.621875,0.519230,0.018944,27.408668,0.349746,0.349746,2.434328,0.012415,0.145015,10044.0


In [44]:
df_aggregate_data.loc[
                     (df_aggregate_data['daily_vessel_id_'].isin([10049
                                                                    ,10073
                                                                    ,10074
                                                                    ,10084
                                                                    ,10149
                                                                    ,10150
                                                                    ,10161
                                                                    ,10174
                                                                    ,10186
                                                                    ,10189
                                                                    ,10206]))]

,hour_since_sum,mile_since_sum,period_total_distance,distance_ratio,fuel_consumption_average_max,fuel_consumption_average_mean,speed_nm_max,speed_nm_median,speed_nm_mean,daily_vessel_id_
48,14.980647,0.598476,0.330414,1.811291,0.102602,0.102602,1.941660,0.003170,0.040531,10049.0
72,8.511111,0.321377,0.315529,1.018533,0.110654,0.110654,0.328280,0.000832,0.033536,10073.0
73,18.281789,0.331872,0.261480,1.269207,0.107262,0.107262,1.701887,0.002633,0.052554,10074.0
83,18.590423,0.401907,0.363264,1.106377,0.098566,0.098566,0.736956,0.003077,0.041703,10084.0
148,12.018258,0.310980,0.235808,1.318782,0.077838,0.077838,0.552879,0.002923,0.027913,10149.0
149,24.061530,6.224930,5.565341,1.118517,0.071250,0.071250,8.969303,0.003265,0.356337,10150.0
160,10.522987,8.326295,4.923127,1.691261,0.082744,0.082744,12.224488,0.003873,2.181922,10161.0
173,16.422654,0.217745,0.212630,1.024057,0.054865,0.054865,2.912386,0.000446,0.427062,10174.0
185,1.901954,0.402529,0.297549,1.352817,0.256457,0.256457,1.172383,0.002237,0.207417,10186.0
188,0.394191,0.120147,0.119079,1.008969,0.170905,0.170905,0.612867,0.047976,0.222016,10189.0


In [45]:
df_aggregate_data.columns

Index(['hour_since_sum', 'mile_since_sum', 'period_total_distance',
       'distance_ratio', 'fuel_consumption_average_max',
       'fuel_consumption_average_mean', 'speed_nm_max', 'speed_nm_median',
       'speed_nm_mean', 'daily_vessel_id_'],
      dtype='object')

In [46]:
# list_label_sequence = ['Port', 'TransitCombine', 'Standby', 'DP', 'Transit Max']
# df_aggregate_data['activity_label_code'] = df_aggregate_data.apply(lambda x: list_label_sequence.index(x.activity_label_first), axis=1)

In [47]:
# df_aggregate_data['activity_label_code'] = df_aggregate_data.activity_label_first.factorize()[0]

In [48]:
test_dataset_x = df_aggregate_data.loc[(df_aggregate_data['speed_nm_max'].isna()==False) , 
                                     ['hour_since_sum', 'mile_since_sum',
       'period_total_distance', 'distance_ratio',
       'fuel_consumption_average_max', 'fuel_consumption_average_mean',
       'speed_nm_max', 'speed_nm_median', 'speed_nm_mean'
                                     ]]


# test_dataset_y = df_aggregate_data.loc[(df_aggregate_data['speed_nm_max'].isna()==False), 'activity_label_code']

In [49]:
test_dataset_x.count()

hour_since_sum                   34243
mile_since_sum                   34243
period_total_distance            34243
distance_ratio                   34243
fuel_consumption_average_max     34243
fuel_consumption_average_mean    34243
speed_nm_max                     34243
speed_nm_median                  34243
speed_nm_mean                    34243
dtype: int64

In [63]:
df_aggregate_data.loc[(df_aggregate_data['speed_nm_max'].isna()==False)].count()

hour_since_sum                   34243
mile_since_sum                   34243
period_total_distance            34243
distance_ratio                   34243
fuel_consumption_average_max     34243
fuel_consumption_average_mean    34243
speed_nm_max                     34243
speed_nm_median                  34243
speed_nm_mean                    34243
daily_vessel_id_                 34243
random_forest_predictions        34243
xgb_predictions                  34243
ensemble_predictions             34243
dtype: int64

In [51]:
# load the model from disk
file_name = '[PSV]random_forest_model.pkl'
randomforrest_clf = pickle.load(open('model/'+file_name, 'rb'))
# randomforrest_score = randomforrest_clf.score(test_dataset_x, test_dataset_y)
rf_predictions = randomforrest_clf.predict(test_dataset_x)
# print(randomforrest_score)

In [52]:
# load the model from disk
file_name = '[PSV]xg_boost_model.pkl'
xgboost_clf = pickle.load(open('model/'+file_name, 'rb'))
# xgboost_score = xgboost_clf.score(test_dataset_x, test_dataset_y)
xgboost_predictions = xgboost_clf.predict(test_dataset_x)
# print(xgboost_score)

In [53]:
# load the model from disk
file_name = '[PSV]ensemble_model3.pkl'
ensemble_clf = pickle.load(open('model/'+file_name, 'rb'))
# ensemble_score = ensemble_clf.score(test_dataset_x, test_dataset_y)
ensemble_predictions = ensemble_clf.predict(test_dataset_x)
# print(ensemble_score)

In [54]:
df_aggregate_data.loc[(df_aggregate_data['speed_nm_max'].isna()==False),'random_forest_predictions'] = rf_predictions
df_aggregate_data.loc[(df_aggregate_data['speed_nm_max'].isna()==False),'xgb_predictions'] = xgboost_predictions
df_aggregate_data.loc[(df_aggregate_data['speed_nm_max'].isna()==False),'ensemble_predictions'] = ensemble_predictions

In [55]:
# df_aggregate_data.loc[(df_aggregate_data['speed_nm_max'].isna()==False), ['activity_label_first', 'activity_label_code', 'random_forest_predictions', 'xgb_predictions']]

In [56]:
df_aggregate_data.to_csv(file_name, sep='\t', encoding='utf-8')

In [57]:
df_aggregate_data.columns
# df_aggregate_data['daily_vessel_id']

Index(['hour_since_sum', 'mile_since_sum', 'period_total_distance',
       'distance_ratio', 'fuel_consumption_average_max',
       'fuel_consumption_average_mean', 'speed_nm_max', 'speed_nm_median',
       'speed_nm_mean', 'daily_vessel_id_', 'random_forest_predictions',
       'xgb_predictions', 'ensemble_predictions'],
      dtype='object')

In [58]:
list_ensemble_predictions = df_aggregate_data['ensemble_predictions'].to_numpy()
list_randomforest_predictions = df_aggregate_data['random_forest_predictions'].to_numpy()

In [59]:
df_vessel_daily_data_raw.loc[df_vessel_daily_data_raw['id'].isin(df_aggregate_data['daily_vessel_id_']), 'prediction_ens'] = list_ensemble_predictions
df_vessel_daily_data_raw.loc[df_vessel_daily_data_raw['id'].isin(df_aggregate_data['daily_vessel_id_']), 'prediction_rf'] = list_randomforest_predictions

In [60]:
df_vessel_daily_data_raw.loc[df_vessel_daily_data_raw['id'].isin(df_aggregate_data['daily_vessel_id_'])].count()

vessel_id         34243
name              34243
type              34243
report_date       34243
report_hours      34243
time_period       34243
time_offset       34243
fuel              34243
activity              0
activitymode          0
id                34243
prediction_ens    34243
prediction_rf     34243
dtype: int64

In [65]:
# transform to activity id
df_vessel_daily_data_raw.loc[df_vessel_daily_data_raw['prediction_ens'] == 0, 'prediction_activity_id_model1'] = 6
df_vessel_daily_data_raw.loc[df_vessel_daily_data_raw['prediction_ens'] == 1, 'prediction_activity_id_model1'] = 7
df_vessel_daily_data_raw.loc[df_vessel_daily_data_raw['prediction_ens'] == 2, 'prediction_activity_id_model1'] = 3
df_vessel_daily_data_raw.loc[df_vessel_daily_data_raw['prediction_ens'] == 3, 'prediction_activity_id_model1'] = 2
df_vessel_daily_data_raw.loc[df_vessel_daily_data_raw['prediction_ens'] == 4, 'prediction_activity_id_model1'] = 9


df_vessel_daily_data_raw.loc[df_vessel_daily_data_raw['prediction_rf'] == 0, 'prediction_activity_id_model2'] = 6
df_vessel_daily_data_raw.loc[df_vessel_daily_data_raw['prediction_rf'] == 1, 'prediction_activity_id_model2'] = 7
df_vessel_daily_data_raw.loc[df_vessel_daily_data_raw['prediction_rf'] == 2, 'prediction_activity_id_model2'] = 3
df_vessel_daily_data_raw.loc[df_vessel_daily_data_raw['prediction_rf'] == 3, 'prediction_activity_id_model2'] = 2
df_vessel_daily_data_raw.loc[df_vessel_daily_data_raw['prediction_rf'] == 4, 'prediction_activity_id_model2'] = 9

In [66]:
df_vessel_daily_data_raw.count()
# df_vessel_daily_data_raw.loc[df_vessel_daily_data_raw['prediction_ens'].isna() == False]


vessel_id                        42033
name                             42033
type                             42033
report_date                      42033
report_hours                     42033
time_period                      42033
time_offset                      42033
fuel                             42033
activity                             0
activitymode                         0
id                               42033
prediction_ens                   34243
prediction_rf                    34243
prediction_activity_id_model1    34243
prediction_activity_id_model2    34243
dtype: int64

In [69]:
df_vessel_daily_data_raw.to_csv('data/test/daily_vessel_data_with_prediction_two_prediction_fixed.csv', encoding='utf-8')

In [53]:
print(datetime.now())

2021-03-08 23:33:42.384155


In [54]:
# df_duplicates.to_csv('data/test/daily_vessel_data_with_prediction_DUPLICATES', encoding='utf-8')

In [55]:
# df_duplicates

In [56]:
# df_prediction_check = pd.read_csv("./data/test/daily_vessel_data_with_prediction (1)", sep=',')

In [57]:
# df_prediction_check.head(20)

In [58]:
# df_vessel_daily_data_raw.head(20)

In [59]:
# df_prediction_check.count()